In [1]:
import pandas as pd
import numpy as np
import subprocess
import glob
import re
import os, sys
from collections import defaultdict, Counter, OrderedDict
from Bio import SeqIO, SeqRecord, Seq
from tRNA_position import *
pd.set_option('display.max_colwidth',10000)
pd.set_option('display.width', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows',1000)
isotypes = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Glu', 'Gly', 'His', 'Ile', 'iMet', 'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val']

# Introduction

A global view of identity elements versus biological features would be a powerful tool for predicting tRNA function using primary sequence. To do this, I'll need to align eukaryotic tRNAs to our established models and annotate positions based on universal numbering. Then, I'll extract sequence information such as position, clade, or variable arm length.

# Process tRNAs
## Species information

In [ ]:
species_table = pd.read_table('genomes-051418.tsv', header=None, names=['species', 'longname', 'domain', 'clade', 'taxid'], dtype={'taxid': str})
species_table.head()

,species,longname,domain,clade,taxid
0,ashbGoss_ATCC10895,Eremothecium gossypii ATCC 10895,eukaryota,Fungi,33169
1,aspeFumi_AF293,Aspergillus fumigatus Af293,eukaryota,Fungi,746128
2,aspeNidu_FGSC_A4,Aspergillus nidulans FGSC A4,eukaryota,Fungi,162425
3,aspeOryz_RIB40,Aspergillus oryzae RIB40,eukaryota,Fungi,5062
4,botrCine_B05_10,Botrytis cinerea B05.10,eukaryota,Fungi,40559


## Align tRNAs to Sprinzl numbering model

We have a "quality set" of tRNAs, which include tRNAs from previous tRNAscan-SE runs and newer runs. We also want to keep track of high-scoring, isotype mismatched tRNAs.

In [ ]:
species = sorted(glob.glob("out/*-detailed.out"))
species = [sp[4:-19] for sp in species]
seqs = []
for sp in species:
  if sp not in species_table.values: continue
  clade = species_table.loc[species_table.species == sp, 'clade'].values[0]
  sys.stdout.write('processing ' + sp + '...')
  sys.stdout.flush()
  
  tRNA_file = 'tRNAs/' + sp + '-tRNAs.fa'
  tscanout_file = 'out/{}-tRNAs-detailed.out'.format(sp)
  # Import annotation from tRNAscan .out files.
  # Filter tRNAs here to speed up data import. We want to keep a small number of lower quality tRNAs in case it's needed for lifting to other isotypes.
  approved_tRNAs = []
  intron_lengths = [] # we are purging introns to improve alignment, so store intron lengths here
  for metadata in pd.read_table(tscanout_file, sep="\t", skiprows=3, na_filter = False, header=None).iterrows():
    if ((clade == 'Fungi' and metadata[1].iloc[-1] == "Isotype mismatch;Isotype mismatch;First-pass quality filtered") or
         metadata[1].iloc[-1] in ["",
                                  "high confidence set",
                                  "secondary filtered",
                                  "tertiary filtered", 
                                  "Quality set",
                                  "unexpected anticodon",
                                  "Unexpected anticodon;First-pass quality filtered",
                                  "Isotype mismatch;First-pass quality filtered",
                                  "First-pass quality filtered",
                                  "Second-pass quality filtered"] or
         re.match('^IPD:-\d+\.\d+(,(isotype mismatch|secondary filtered|tertiary filtered|unexpected anticodon))?$', metadata[1].iloc[-1]) or
         (clade == 'Fungi' and 'cryp' in sp and metadata[1].iloc[4] == 'Asp' and float(metadata[1].iloc[8]) > 35)):
      if metadata[1][4] == 'Met' and metadata[1][12] == 'iMet': isotype = 'iMet'
      else: isotype = metadata[1][4]
      approved_tRNAs.append('{}.trna{}-{}{}'.format(str(metadata[1][0]).strip(), metadata[1][1], isotype, metadata[1][5]))
      intron_length = abs(int(metadata[1][6]) - int(metadata[1][7]))
      if intron_length > 0: intron_length = intron_length + 1
      intron_lengths.append(intron_length)
  # Remove introns using sstofa
  subprocess.call('sstofa3 ss/{}-tRNAs.ss "" 1 0 > {}'.format(sp, tRNA_file), shell=True)

  # Parse isotype-specific scores file
  iso_scores = pd.read_table('iso/' + sp + '-tRNAs.iso', header=0)
  iso_scores['Undet'] = 0
  iso_scores['Sup'] = 0
  iso_scores['best'] = iso_scores.iloc[:,2:].idxmax(axis=1)
  iso_scores['score'] = iso_scores.max(axis=1, numeric_only=True)
  iso_scores['ac_score'] = iso_scores.lookup(iso_scores.index, iso_scores.iloc[:, 1])
  iso_scores.index = iso_scores.tRNAscanID.values
  iso_scores = iso_scores[['best', 'score', 'ac_score']]
  
  # Parse seqs, filtering out the ones we don't need
  for seq in SeqIO.parse('tRNAs/{}-tRNAs.fa'.format(sp), 'fasta'):
    if seq.id not in approved_tRNAs:
      seq.id = seq.id.replace('Met', 'iMet')
      if seq.id not in approved_tRNAs: continue
    if "pseudogene" in seq.description and 'cryp' not in sp: continue
    intron_length = intron_lengths[approved_tRNAs.index(seq.id)]
    score = float(re.findall('Sc: [\d\.]+', seq.description)[0].split()[-1])
    # relax tRNA score requirements for fungi
    if (species_table[species_table.species == sp].clade.values[0] == "Fungi"):
      if score < 30 and seq.description != 'chrI.trna9-MetCAT (3515186-3515314)  Met (CAT) 70 bp  Sc: 27.6': continue
    elif species_table[species_table.species == sp].clade.values[0] == "Insect":
      if score < 50: continue
    else:
      if score < 55: continue
    trnascanid = re.findall('.+\.trna\d+', seq.id)[0]
    best_isotype = iso_scores.loc[trnascanid].best
    if best_isotype == 'SeC' or 'mito' in best_isotype or 'mt' in trnascanid: continue
    isoscore, ac_score = iso_scores.loc[trnascanid].score, iso_scores.loc[trnascanid].ac_score
    seq.id = '{}|{} Iso: {} ({}) Iso_ac: {} Intron: {}'.format(sp, seq.description, isoscore, best_isotype, ac_score, intron_length)
    seq.description = ''
    seqs.append(seq)

  print('finished')
  sys.stdout.flush()

processing ailMel1...finished
processing anoCar2...finished
processing anoGam2...finished
processing apiMel1...finished
processing araTha1...finished
processing ashbGoss_ATCC10895...finished
processing aspeFumi_AF293...finished
processing aspeNidu_FGSC_A4...finished
processing aspeOryz_RIB40...finished
processing balAcu1...finished
processing bomTer1...finished
processing bosTau8...finished
processing botrCine_B05_10...finished
processing braDis2...finished
processing braOle1...finished
processing bruMal1...finished
processing caeJap1...finished
processing caePb2...finished
processing caeRem3...finished
processing calJac3...finished
processing calMil1...finished
processing canFam3...finished
processing candAlbi_WO_1...finished
processing candDubl_CD36...finished
processing candGlab_CBS_138...finished
processing candOrth_CO_90_125...finished
processing carPap1...finished
processing cavPor3...finished
processing cb3...finished
processing ce11...finished
processing cerSim1...finished
proc

In [ ]:
fasta_handle = open('euk-tRNAs.fa', 'w')
SeqIO.write(seqs, fasta_handle, 'fasta')
fasta_handle.close()
num_model = '/projects/lowelab/users/blin/tRNAscan/models/domain-specific/euk-num-092016.cm'
subprocess.call('cmalign -g --notrunc -o euk-tRNAs.sto {} euk-tRNAs.fa'.format(num_model), shell=True)

## Create table of tRNA bases by position

This is a giant data frame with one row per tRNA, and with columns for each position, plus tRNA metadata like species and loop lengths.

In [ ]:
def position_base(positions, seq):
  for position_index, position in enumerate(positions):
    if position.paired:
      index1, index2 = position.position.split(':')
      index1, index2 = int(index1), int(index2)
      base_pair = "{}:{}".format(seq[index1 - 1].upper(), seq[index2 - 1].upper())
      yield position.sprinzl, base_pair
    else:
      index = int(position.position)
      base = seq[index - 1].upper()
      yield position.sprinzl, base
      
identities = pd.DataFrame()

# get positions
alignment_fhandle = open('euk-tRNAs.sto')
positions = [] # list containing each position in the tRNA

# first, get secondary structure
for line in alignment_fhandle:
  if line[0:12] == '#=GC SS_cons':
    ss = line.strip().split()[-1]
alignment_fhandle.close()

# parse secondary structure into regions and positions
positions = annotate_positions(ss)

# get nucleotide at each position for each tRNA by parsing Stockholm file
alignment_fhandle = open('euk-tRNAs.sto')
trnas = []
skipped = []
for line in alignment_fhandle:
  if line[0] in ["#", '\n', '/']: continue
  species, desc = line.strip().split('|', 1)
  seqname = desc.split()[0]
  seq = desc.split()[-1]
  if any(species_table.species == species):
    row = species_table[species_table.species == species]
  else: 
    skipped.append(species)
    continue
  isotype = re.findall('\.trna\d+-([A-Za-z]+)', seqname)[0][:-3]
  seqname = '{}_{}'.format(species, seqname)
  trna = {'domain': row.domain.values[0], 'clade': row.clade.values[0], 'species': species, 'species_long': row.longname.values[0], 'taxid': row.taxid.values[0], 'seqname': seqname, 'isotype': isotype}
  trna = {**trna, **{sprinzl: base for sprinzl, base in position_base(positions, seq)}}
  trnas.append(trna)

if len(skipped) > 0: print('skipped the following: {}'.format(set(skipped)))
identities = identities.append(trnas, ignore_index=True)
identities.fillna('.', inplace=True)
alignment_fhandle.close()

### Create single base columns from paired positions

This is mainly for plotting and such.

In [ ]:
cols = list(filter(lambda x: ':' in x, identities.columns))
for col in cols:
  pos1, pos2 = col.split(':')
  base1 = [bases.split(':')[0] for bases in identities[col]]
  base2 = [bases.split(':')[1] for bases in identities[col]]
  identities[pos1] = base1
  identities[pos2] = base2

### Annotate tertiary interactions

We also want to annotate the 2-base version of the base triples. This makes checking for conserved interactions (e.g. an identity element) a bit easier. It's important to remember to consider the 3rd base pair too, though.

In [ ]:
identities['8:14:21'] = identities.loc[:, ["8", "14", "21"]].apply(lambda row: ':'.join(row), axis=1)
identities['8:14'] = identities.loc[:, ["8", "14"]].apply(lambda row: ':'.join(row), axis=1)
identities['9:12:23'] = identities.loc[:, ["9", "12", "23"]].apply(lambda row: ':'.join(row), axis=1)
identities['9:23'] = identities.loc[:, ["9", "23"]].apply(lambda row: ':'.join(row), axis=1)
identities['10:25:45'] = identities.loc[:, ["10", "25", "45"]].apply(lambda row: ':'.join(row), axis=1)
identities['10:45'] = identities.loc[:, ["10", "45"]].apply(lambda row: ':'.join(row), axis=1)
identities['13:22:46'] = identities.loc[:, ["13", "22", "46"]].apply(lambda row: ':'.join(row), axis=1)
identities['22:46'] = identities.loc[:, ["22", "46"]].apply(lambda row: ':'.join(row), axis=1)
identities['15:48'] = identities.loc[:, ["15", "48"]].apply(lambda row: ':'.join(row), axis=1)
identities['18:55'] = identities.loc[:, ["18", "55"]].apply(lambda row: ':'.join(row), axis=1)
identities['19:56'] = identities.loc[:, ["19", "56"]].apply(lambda row: ':'.join(row), axis=1)
identities['26:44'] = identities.loc[:, ["26", "44"]].apply(lambda row: ':'.join(row), axis=1)
identities['54:58'] = identities.loc[:, ["54", "58"]].apply(lambda row: ':'.join(row), axis=1)

### Additional sequence information

In [ ]:
# Basic information - isotype, anticodon, score
seqinfo = []
for line in open('euk-tRNAs.sto'):
  if line[0:4] != "#=GS": continue
  _, desc, _, _, isotype, anticodon, _, _, _, score, _, isoscore, isotype_best, _, isoscore_ac, _, intron_length = line.strip().split()
  seqname = desc.replace('|', '_', 1)
  seqinfo.append([seqname, isotype_best[1:-1], anticodon[1:-1], float(score), float(isoscore), float(isoscore_ac), int(intron_length)])
seqinfo = pd.DataFrame(seqinfo, columns=['seqname', 'isotype_best', 'anticodon', 'score', 'isoscore', 'isoscore_ac', 'intron'])
identities = identities.merge(seqinfo, on='seqname')

# GC content
paired_cols = identities.columns[list(map(lambda x: (':' in x), identities.columns))]
identities['GC'] = identities[paired_cols].apply(lambda x: sum((x == "G:C") | (x == "C:G"))/len(paired_cols), axis=1)

### Adjust isotype designations

In [ ]:
# iMet may or may not be properly annotated, depending on tRNAscan-SE run
unlifted_imets = identities[(identities.isotype == "Met") & (identities.isotype_best == "iMet")].index
identities.loc[unlifted_imets, 'isotype'] = 'iMet'

identities['adjusted'] = False
# Manage tRNAs of missing isotypes
for sp in identities.species.unique():
  # Lift the highest-scoring iMet if species does not contain iMet
  if 'iMet' not in identities[identities.species == sp].isotype.values:
    # reparse iso file and re-assign the Met with the highest iMet score
    imet_scores = {}
    seqname_isotypes = {}
    for line in open('iso/{}-tRNAs.iso'.format(sp)):
      tabs = line.strip().split('\t')
      if tabs[0] == 'tRNAscanID': continue
      seqname = sp + '_' + tabs[0]
      imet_scores[seqname] = float(tabs[-1])
      seqname_isotypes[seqname] = tabs[1]
    
    best_seqname = max(imet_scores, key = imet_scores.get)
    best_score = np.max(imet_scores.values())
    print('{}: adjusting {} to iMet'.format(sp, identities.loc[identities.seqname.apply(lambda x: x[:-7]) == best_seqname, 'seqname'].values.tolist()))
    identities.loc[identities.seqname.apply(lambda x: x[:-7]) == best_seqname, 'isotype'] = 'iMet'
    identities.loc[identities.seqname.apply(lambda x: x[:-7]) == best_seqname, 'adjusted'] = True
    for seqname in imet_scores:
      if abs(imet_scores[seqname] - imet_scores[best_seqname]) < 2 and seqname_isotypes[seqname] == seqname_isotypes[best_seqname]:
        print('{}: adjusting {} to iMet'.format(sp, identities.loc[identities.seqname.apply(lambda x: x[:-7]) == seqname, 'seqname'].values.tolist()))
        identities.loc[identities.seqname.apply(lambda x: x[:-7]) == seqname, 'isotype'] = 'iMet'
        identities.loc[identities.seqname.apply(lambda x: x[:-7]) == seqname, 'adjusted'] = True

  # Revert tRNAs in species that are missing tRNAs of an isotype
  for isotype in isotypes:
    if isotype not in identities.loc[identities.species == sp].isotype.unique():
      if identities.loc[(identities.species == sp) & (identities.seqname.apply(lambda x: isotype in x))].seqname.size > 0:
        print('{}: adjusting {} to {}'.format(sp, identities.loc[(identities.species == sp) & (identities.seqname.apply(lambda x: isotype in x)), 'seqname'].values.tolist(), isotype))
        identities.loc[(identities.species == sp) & (identities.seqname.apply(lambda x: isotype in x)), 'isotype'] = isotype
        identities.loc[(identities.species == sp) & (identities.seqname.apply(lambda x: isotype in x)), 'adjusted'] = True
      else:
        print('{}: missing {} tRNAs'.format(sp, isotype))

### Insertions/deletions

In [ ]:
# Insertions (minus misaligned introns at 37/38)
intron_cols = list(filter(lambda x: x[0:3] == '37i', identities.columns))
insertion_cols = list(filter(lambda x: bool(re.search('^\d+i', x)) & (x not in intron_cols), identities.columns))
identities['insertions'] = identities[insertion_cols].apply(lambda x: sum(x != '.'), axis=1)

# Deletions at positions that are not the variable arm, and not counting 17/17a/20a/20b
base_cols = list(filter(lambda x: bool(re.match('^\d+$', x)) & (x not in ['74', '75', '76', '17', '17a', '20a', '20b']), identities.columns))
identities['deletions'] = identities[base_cols].apply(lambda x: ''.join(x).count('-'), axis=1)

### Loop sizes

In [ ]:
def bounds_to_cols(cols, start, end):
  selected_cols = []
  for col in cols:
    matches = re.findall('\d+', col)
    if len(matches) < 1: continue
    index = int(matches[0])
    if (index >= start and index <= end or col[0:3] == '{}i'.format(start - 1)) and col[0] != 'V':
      selected_cols.append(col)
  return selected_cols

dloop_cols = list(filter(lambda col: ':' not in col, bounds_to_cols(identities.columns, 14, 21)))
identities['D-loop'] = identities[dloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

# Leu, Ser have a 3 bp D stem
dloop_II_cols = list(filter(lambda col: ':' not in col, bounds_to_cols(identities.columns, 13, 22)))
identities.loc[(identities.isotype == 'Leu') | (identities.isotype == 'Ser'), 'D-loop'] = identities[dloop_II_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

acloop_cols = list(filter(lambda x: not re.match('37i.+', x), bounds_to_cols(identities.columns, 32, 38)))
identities['AC-loop'] = identities[acloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

tpcloop_cols = bounds_to_cols(identities.columns, 54, 60)
identities['TPC-loop'] = identities[tpcloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

varm_cols = list(filter(lambda x: ('V' in x) & (':' not in x) | (x in bounds_to_cols(identities.columns, 45, 48)), identities.columns))
identities['V-arm'] = identities[varm_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

### Restrict tRNAs by species

We may also want to limit the contribution of any single species, similar to how we built the isotype-specific models. tRNAs that are restricted are excluded from analysis. We'll start from the confidence set if applicable and add in unique tRNAs from there.

In [ ]:
# Import list of quality tRNAs
confidence_trnas = [line.strip() for line in open('confidence-set.txt')]
identities['confidence'] = identities.seqname.isin(confidence_trnas)

identities['restrict'] = True
identities['duplicate'] = True

identities.loc[identities.adjusted, 'restrict'] = False
identities.loc[identities.score > 45, 'restrict'] = False

for species in identities.species.unique():
  for isotype in isotypes:
    isotype_indices = identities.loc[(identities.species == species) & (identities.isotype == isotype) & (-identities.restrict)].index
    confidence_indices = identities.loc[(identities.species == species) & (identities.isotype == isotype) & (-identities.restrict) & (identities.confidence)].index
    unique_scores_indices = isotype_indices[-identities.iloc[isotype_indices].score.duplicated()]
    unique_confidence_indices = confidence_indices[-identities.iloc[confidence_indices].score.duplicated()]
    
    # no tRNAs means this is a fungi missing an isotype, lower score threshold to 30 and skip rest
    if len(unique_scores_indices) == 0: 
      isotype_indices = identities.loc[(identities.species == species) & (identities.isotype == isotype)].index
      if (len(isotype_indices) == 0): continue
      unique_scores_indices = isotype_indices[-identities.iloc[isotype_indices].score.duplicated()]
      identities.loc[unique_scores_indices, 'duplicate'] = False
      sorted_unique_indices = unique_scores_indices[np.argsort(identities.loc[unique_scores_indices, 'score'])]
      identities.loc[sorted_unique_indices[-1], 'adjusted'] = True
      identities.loc[sorted_unique_indices[-1], 'restrict'] = False
      identities.loc[sorted_unique_indices[:-1], 'restrict'] = True
      continue
      
    identities.loc[unique_scores_indices, 'duplicate'] = False
    
    if len(unique_confidence_indices) >= 50: # too many unique tRNAs; likely repetitive
      unique_confidence_indices = unique_confidence_indices[np.argsort(identities.loc[unique_confidence_indices, 'score'])][::-1][:50]
      restricted_indices = list(set(isotype_indices) - set(unique_confidence_indices))
    elif len(unique_confidence_indices) > 0:
      if len(unique_scores_indices) > 50 - len(unique_confidence_indices):
        unique_scores_indices = unique_scores_indices[np.argsort(identities.loc[unique_scores_indices, 'score'])][::-1][:50 - len(unique_confidence_indices)]
      restricted_indices = list(set(isotype_indices) - set(unique_confidence_indices) - set(unique_scores_indices))
    else:
      if len(unique_scores_indices) > 50:
        unique_scores_indices = unique_scores_indices[np.argsort(identities.loc[unique_scores_indices, 'score'])][::-1][:50]
      restricted_indices = list(set(isotype_indices) - set(unique_scores_indices))
    
    identities.loc[restricted_indices, 'restrict'] = True

identities.loc[identities.duplicate, 'restrict'] = True

The quality set does have some issues, though: it filters out rare codons, when rare is relative to model species. It also performs a hard check for anticodon-isotype model match. Fungi have much more diverged tRNAs and, in many cases, a different isotype scores higher even when it's clear that it's the bona fide tRNA (mostly based on number of tRNAs of that isotype, or codon usage). 

## Export to R

R has superior visualization capabilities.

### Order columns

To make it look pretty.

In [ ]:
def position_str_to_int(position):
  if position == "20a": return 20.1
  if position == "20b": return 20.2
  digits = re.findall('\d+', position)
  if len(digits) == 0: return -1
  insert = 0
  if 'i' in position and len(digits) == 2: insert = float(digits[1]) / 1000
  if position[0] == 'V':
    if ':' in position: return int(digits[0]) + 45 - 10 + insert # V11~V17
    else: return int(digits[0]) + 45 + 7 + insert # V1~V5
  if int(digits[0]) >= 46: return int(digits[0]) + 50 + insert # just add an arbitrarily large number to skip v-arm
  return int(digits[0]) + insert

identities = identities[sorted(list(identities.columns), key=position_str_to_int)]

In [ ]:
identities.to_csv(path_or_buf='identities.tsv', sep='\t', index_label=False, index=False)

In [ ]:
# Export fasta of tRNAs used for analysis
fasta_handle = open('euk-tRNAs-filtered.fa', 'w')
filtered_seqs = []
seqnames = identities[~identities.restrict].seqname.tolist()
for seq in SeqIO.parse('euk-tRNAs.fa', 'fasta'):
  seqname = seq.name.replace('|', '_')
  if seqname in seqnames: filtered_seqs.append(seq)
SeqIO.write(filtered_seqs, fasta_handle, 'fasta')
fasta_handle.close()

In [ ]:
identities.head()